In [1]:
import sys
sys.path.append('..')
import numpy as np
from common.layers import Embedding
from negative_sampling_layer import NegativeSamplingLoss

In [2]:
class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size
        
        W_in = .01 * np.random.randn(V, H).astype('f')
        W_out = .01 * np.random.randn(V, H).astype('f')
        
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=.75, sample_size=5)
        
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads
            
        self.word_vecs = W_in
        
    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i])
        h *= 1 / len(self.in_layers)
        loss = self.ns_loss.forward(h, target)
        return loss
    
    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None

In [3]:
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from common.util import create_contexts_target, to_cpu
from dataset import ptb

In [4]:
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

In [5]:
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

In [6]:
contexts, target = create_contexts_target(corpus, window_size)

In [7]:
model = CBOW(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

In [8]:
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

| 에폭 1 |  반복 1 / 9295 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 21 / 9295 | 시간 1[s] | 손실 4.16
| 에폭 1 |  반복 41 / 9295 | 시간 2[s] | 손실 4.15
| 에폭 1 |  반복 61 / 9295 | 시간 3[s] | 손실 4.13
| 에폭 1 |  반복 81 / 9295 | 시간 4[s] | 손실 4.05
| 에폭 1 |  반복 101 / 9295 | 시간 6[s] | 손실 3.93
| 에폭 1 |  반복 121 / 9295 | 시간 7[s] | 손실 3.78
| 에폭 1 |  반복 141 / 9295 | 시간 8[s] | 손실 3.64
| 에폭 1 |  반복 161 / 9295 | 시간 9[s] | 손실 3.48
| 에폭 1 |  반복 181 / 9295 | 시간 10[s] | 손실 3.38
| 에폭 1 |  반복 201 / 9295 | 시간 11[s] | 손실 3.25
| 에폭 1 |  반복 221 / 9295 | 시간 13[s] | 손실 3.15
| 에폭 1 |  반복 241 / 9295 | 시간 14[s] | 손실 3.07
| 에폭 1 |  반복 261 / 9295 | 시간 15[s] | 손실 3.01
| 에폭 1 |  반복 281 / 9295 | 시간 16[s] | 손실 2.97
| 에폭 1 |  반복 301 / 9295 | 시간 17[s] | 손실 2.92
| 에폭 1 |  반복 321 / 9295 | 시간 18[s] | 손실 2.88
| 에폭 1 |  반복 341 / 9295 | 시간 19[s] | 손실 2.86
| 에폭 1 |  반복 361 / 9295 | 시간 21[s] | 손실 2.81
| 에폭 1 |  반복 381 / 9295 | 시간 22[s] | 손실 2.79
| 에폭 1 |  반복 401 / 9295 | 시간 23[s] | 손실 2.78
| 에폭 1 |  반복 421 / 9295 | 시간 24[s] | 손실 2.71
| 에폭 1 |  반복 441 / 9295 |

| 에폭 1 |  반복 3561 / 9295 | 시간 206[s] | 손실 2.40
| 에폭 1 |  반복 3581 / 9295 | 시간 207[s] | 손실 2.40
| 에폭 1 |  반복 3601 / 9295 | 시간 208[s] | 손실 2.40
| 에폭 1 |  반복 3621 / 9295 | 시간 209[s] | 손실 2.43
| 에폭 1 |  반복 3641 / 9295 | 시간 211[s] | 손실 2.39
| 에폭 1 |  반복 3661 / 9295 | 시간 212[s] | 손실 2.43
| 에폭 1 |  반복 3681 / 9295 | 시간 213[s] | 손실 2.43
| 에폭 1 |  반복 3701 / 9295 | 시간 214[s] | 손실 2.41
| 에폭 1 |  반복 3721 / 9295 | 시간 215[s] | 손실 2.41
| 에폭 1 |  반복 3741 / 9295 | 시간 216[s] | 손실 2.44
| 에폭 1 |  반복 3761 / 9295 | 시간 218[s] | 손실 2.40
| 에폭 1 |  반복 3781 / 9295 | 시간 219[s] | 손실 2.41
| 에폭 1 |  반복 3801 / 9295 | 시간 220[s] | 손실 2.39
| 에폭 1 |  반복 3821 / 9295 | 시간 221[s] | 손실 2.38
| 에폭 1 |  반복 3841 / 9295 | 시간 222[s] | 손실 2.44
| 에폭 1 |  반복 3861 / 9295 | 시간 223[s] | 손실 2.42
| 에폭 1 |  반복 3881 / 9295 | 시간 225[s] | 손실 2.38
| 에폭 1 |  반복 3901 / 9295 | 시간 226[s] | 손실 2.40
| 에폭 1 |  반복 3921 / 9295 | 시간 227[s] | 손실 2.42
| 에폭 1 |  반복 3941 / 9295 | 시간 228[s] | 손실 2.41
| 에폭 1 |  반복 3961 / 9295 | 시간 229[s] | 손실 2.40
| 에폭 1 |  반복 

| 에폭 1 |  반복 7061 / 9295 | 시간 406[s] | 손실 2.30
| 에폭 1 |  반복 7081 / 9295 | 시간 407[s] | 손실 2.26
| 에폭 1 |  반복 7101 / 9295 | 시간 408[s] | 손실 2.27
| 에폭 1 |  반복 7121 / 9295 | 시간 409[s] | 손실 2.29
| 에폭 1 |  반복 7141 / 9295 | 시간 410[s] | 손실 2.30
| 에폭 1 |  반복 7161 / 9295 | 시간 411[s] | 손실 2.30
| 에폭 1 |  반복 7181 / 9295 | 시간 412[s] | 손실 2.25
| 에폭 1 |  반복 7201 / 9295 | 시간 414[s] | 손실 2.26
| 에폭 1 |  반복 7221 / 9295 | 시간 415[s] | 손실 2.24
| 에폭 1 |  반복 7241 / 9295 | 시간 416[s] | 손실 2.28
| 에폭 1 |  반복 7261 / 9295 | 시간 417[s] | 손실 2.29
| 에폭 1 |  반복 7281 / 9295 | 시간 418[s] | 손실 2.27
| 에폭 1 |  반복 7301 / 9295 | 시간 419[s] | 손실 2.27
| 에폭 1 |  반복 7321 / 9295 | 시간 420[s] | 손실 2.27
| 에폭 1 |  반복 7341 / 9295 | 시간 422[s] | 손실 2.26
| 에폭 1 |  반복 7361 / 9295 | 시간 423[s] | 손실 2.28
| 에폭 1 |  반복 7381 / 9295 | 시간 424[s] | 손실 2.28
| 에폭 1 |  반복 7401 / 9295 | 시간 425[s] | 손실 2.25
| 에폭 1 |  반복 7421 / 9295 | 시간 426[s] | 손실 2.30
| 에폭 1 |  반복 7441 / 9295 | 시간 427[s] | 손실 2.30
| 에폭 1 |  반복 7461 / 9295 | 시간 428[s] | 손실 2.25
| 에폭 1 |  반복 

| 에폭 2 |  반복 1281 / 9295 | 시간 605[s] | 손실 2.17
| 에폭 2 |  반복 1301 / 9295 | 시간 606[s] | 손실 2.14
| 에폭 2 |  반복 1321 / 9295 | 시간 607[s] | 손실 2.11
| 에폭 2 |  반복 1341 / 9295 | 시간 608[s] | 손실 2.16
| 에폭 2 |  반복 1361 / 9295 | 시간 609[s] | 손실 2.13
| 에폭 2 |  반복 1381 / 9295 | 시간 610[s] | 손실 2.14
| 에폭 2 |  반복 1401 / 9295 | 시간 612[s] | 손실 2.14
| 에폭 2 |  반복 1421 / 9295 | 시간 613[s] | 손실 2.14
| 에폭 2 |  반복 1441 / 9295 | 시간 614[s] | 손실 2.13
| 에폭 2 |  반복 1461 / 9295 | 시간 615[s] | 손실 2.16
| 에폭 2 |  반복 1481 / 9295 | 시간 616[s] | 손실 2.16
| 에폭 2 |  반복 1501 / 9295 | 시간 617[s] | 손실 2.16
| 에폭 2 |  반복 1521 / 9295 | 시간 618[s] | 손실 2.13
| 에폭 2 |  반복 1541 / 9295 | 시간 619[s] | 손실 2.13
| 에폭 2 |  반복 1561 / 9295 | 시간 621[s] | 손실 2.20
| 에폭 2 |  반복 1581 / 9295 | 시간 622[s] | 손실 2.12
| 에폭 2 |  반복 1601 / 9295 | 시간 623[s] | 손실 2.13
| 에폭 2 |  반복 1621 / 9295 | 시간 624[s] | 손실 2.15
| 에폭 2 |  반복 1641 / 9295 | 시간 625[s] | 손실 2.16
| 에폭 2 |  반복 1661 / 9295 | 시간 626[s] | 손실 2.16
| 에폭 2 |  반복 1681 / 9295 | 시간 627[s] | 손실 2.14
| 에폭 2 |  반복 

| 에폭 2 |  반복 4781 / 9295 | 시간 803[s] | 손실 2.09
| 에폭 2 |  반복 4801 / 9295 | 시간 804[s] | 손실 2.07
| 에폭 2 |  반복 4821 / 9295 | 시간 805[s] | 손실 2.11
| 에폭 2 |  반복 4841 / 9295 | 시간 806[s] | 손실 2.08
| 에폭 2 |  반복 4861 / 9295 | 시간 807[s] | 손실 2.05
| 에폭 2 |  반복 4881 / 9295 | 시간 809[s] | 손실 2.07
| 에폭 2 |  반복 4901 / 9295 | 시간 810[s] | 손실 2.05
| 에폭 2 |  반복 4921 / 9295 | 시간 811[s] | 손실 2.08
| 에폭 2 |  반복 4941 / 9295 | 시간 812[s] | 손실 2.09
| 에폭 2 |  반복 4961 / 9295 | 시간 813[s] | 손실 2.10
| 에폭 2 |  반복 4981 / 9295 | 시간 814[s] | 손실 2.09
| 에폭 2 |  반복 5001 / 9295 | 시간 815[s] | 손실 2.08
| 에폭 2 |  반복 5021 / 9295 | 시간 817[s] | 손실 2.11
| 에폭 2 |  반복 5041 / 9295 | 시간 818[s] | 손실 2.05
| 에폭 2 |  반복 5061 / 9295 | 시간 819[s] | 손실 2.06
| 에폭 2 |  반복 5081 / 9295 | 시간 820[s] | 손실 2.07
| 에폭 2 |  반복 5101 / 9295 | 시간 821[s] | 손실 2.04
| 에폭 2 |  반복 5121 / 9295 | 시간 822[s] | 손실 2.04
| 에폭 2 |  반복 5141 / 9295 | 시간 823[s] | 손실 2.04
| 에폭 2 |  반복 5161 / 9295 | 시간 825[s] | 손실 2.07
| 에폭 2 |  반복 5181 / 9295 | 시간 826[s] | 손실 2.07
| 에폭 2 |  반복 

| 에폭 2 |  반복 8281 / 9295 | 시간 1003[s] | 손실 2.08
| 에폭 2 |  반복 8301 / 9295 | 시간 1004[s] | 손실 2.04
| 에폭 2 |  반복 8321 / 9295 | 시간 1006[s] | 손실 2.02
| 에폭 2 |  반복 8341 / 9295 | 시간 1007[s] | 손실 2.03
| 에폭 2 |  반복 8361 / 9295 | 시간 1008[s] | 손실 2.06
| 에폭 2 |  반복 8381 / 9295 | 시간 1009[s] | 손실 2.04
| 에폭 2 |  반복 8401 / 9295 | 시간 1010[s] | 손실 2.02
| 에폭 2 |  반복 8421 / 9295 | 시간 1011[s] | 손실 2.05
| 에폭 2 |  반복 8441 / 9295 | 시간 1013[s] | 손실 2.02
| 에폭 2 |  반복 8461 / 9295 | 시간 1014[s] | 손실 2.03
| 에폭 2 |  반복 8481 / 9295 | 시간 1015[s] | 손실 2.06
| 에폭 2 |  반복 8501 / 9295 | 시간 1016[s] | 손실 2.00
| 에폭 2 |  반복 8521 / 9295 | 시간 1017[s] | 손실 2.04
| 에폭 2 |  반복 8541 / 9295 | 시간 1018[s] | 손실 2.08
| 에폭 2 |  반복 8561 / 9295 | 시간 1019[s] | 손실 2.07
| 에폭 2 |  반복 8581 / 9295 | 시간 1021[s] | 손실 2.02
| 에폭 2 |  반복 8601 / 9295 | 시간 1022[s] | 손실 2.02
| 에폭 2 |  반복 8621 / 9295 | 시간 1023[s] | 손실 2.04
| 에폭 2 |  반복 8641 / 9295 | 시간 1024[s] | 손실 2.07
| 에폭 2 |  반복 8661 / 9295 | 시간 1025[s] | 손실 2.01
| 에폭 2 |  반복 8681 / 9295 | 시간 1027[s] | 

KeyboardInterrupt: 

In [9]:
word_vecs = model.word_vecs
params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word

pkl_file = 'cbow_params.pkl'

with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)

In [10]:
from common.util import most_similar

pkl_file = 'cbow_params.pkl'

In [11]:
with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']
    
querys = ['you', 'year', 'car', 'pfizer']
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)


[query] you
 we: 0.6103515625
 someone: 0.59130859375
 i: 0.55419921875
 something: 0.48974609375
 anyone: 0.47314453125

[query] year
 month: 0.71875
 week: 0.65234375
 spring: 0.62744140625
 summer: 0.6259765625
 decade: 0.603515625

[query] car
 luxury: 0.497314453125
 arabia: 0.47802734375
 auto: 0.47119140625
 disk-drive: 0.450927734375
 travel: 0.4091796875

[query] pfizer
 livestock: 0.53076171875
 commodore: 0.52490234375
 itt: 0.50390625
 innopac: 0.490966796875
 interpublic: 0.470947265625


In [12]:
from common.util import analogy

In [13]:
analogy('man', 'king', 'woman', word_to_id, id_to_word, word_vecs, top=5)


[analogy] man:king = woman:?
 she: 4.1796875
 moody: 4.1328125
 share: 4.05078125
 character: 3.966796875
 chain: 3.912109375


In [14]:
from eval import *


[query] you
 we: 0.6103515625
 someone: 0.59130859375
 i: 0.55419921875
 something: 0.48974609375
 anyone: 0.47314453125

[query] year
 month: 0.71875
 week: 0.65234375
 spring: 0.62744140625
 summer: 0.6259765625
 decade: 0.603515625

[query] car
 luxury: 0.497314453125
 arabia: 0.47802734375
 auto: 0.47119140625
 disk-drive: 0.450927734375
 travel: 0.4091796875

[query] toyota
 ford: 0.55078125
 instrumentation: 0.509765625
 mazda: 0.49365234375
 bethlehem: 0.47509765625
 nissan: 0.474853515625
--------------------------------------------------

[analogy] king:man = queen:?
 woman: 5.16015625
 veto: 4.9296875
 ounce: 4.69140625
 earthquake: 4.6328125
 successor: 4.609375

[analogy] take:took = go:?
 went: 4.55078125
 points: 4.25
 began: 4.09375
 comes: 3.98046875
 oct.: 3.90625

[analogy] car:cars = child:?
 children: 5.21875
 average: 4.7265625
 yield: 4.20703125
 cattle: 4.1875
 priced: 4.1796875

[analogy] good:better = bad:?
 more: 6.6484375
 less: 6.0625
 rather: 5.21875
 slow